In [17]:
import json
import glob
from ipywidgets import *
import matplotlib.pyplot as plt
import cv2


def load_data():
    with open('annotaion.json') as f:
        data = json.loads(f.read())
    return data


def save_data():
    with open('annotaion.json', 'w') as f:
        f.write(json.dumps(data, sort_keys=True, indent=4))

data = load_data()
node = 104
component = 'holder'
roi = data[str(node)][component+'_roi']

for dataset_name in data[str(node)][component]:
    dataset = data[str(node)][component][dataset_name]

    @interact(zero=widgets.IntText(min=0, max=399, step=1, value=dataset['zero']))
    def display_set(zero):
            dataset = data[str(node)][component][dataset_name]
            path = '../../dataset/%s_%d_%s_192.168.44.%d' % (component, node, dataset_name, node)
            files = glob.glob(path + '/*.png')

            zero_filenames = [filename for filename in files if int(filename.split('/')[-1].split('_')[0]) == zero]


            x0 = roi['x0']
            x1 = roi['x0'] + roi['dx']
            y0 = roi['y0']
            y1 = roi['y0'] + roi['dy']

            fig, axs = plt.subplots(ncols=3, nrows=1, figsize = (18,5))
            fig.suptitle(' | '.join([component, str(node), dataset_name]), size=16)
            for index, filename in enumerate(zero_filenames):
                image = cv2.imread(filename)[y0:y1,x0:x1,::-1]
                if component == 'holder':
                    midpoint = int(image.shape[1] / 2)
                    image[:,midpoint:midpoint+1,0] = 255
                    image[:,midpoint:midpoint+1,1] = 0
                    image[:,midpoint:midpoint+1,2] = 0
                elif component == 'dosing':
                    midpoint = int(image.shape[0] / 2)
                    image[midpoint:midpoint+1,:,0] = 255
                    image[midpoint:midpoint+1,:,1] = 0
                    image[midpoint:midpoint+1,:,2] = 0

                axs[index].imshow(image)
                if dataset['zero'] != zero:
                    dataset['zero'] = zero
                    save_data()



interactive(children=(IntText(value=325, description='zero'), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntText(value=92, description='zero'), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntText(value=4, description='zero'), Output()), _dom_classes=('widget-interact',))

In [15]:
import annotaion as a
data = a.aligning
save_data()